This notebook provides code to join station data stored in the National Transit Atlas Dataset and the National Transit Database dataset on the basis of station names using Fuzzy Matching to reconcile discrepencies in the name configurations. Additional information on these operations can be found at https://www.transitorienteddiscoveries.com/blog.

# **Join NTA and NTD Datasets on Station Name Using Fuzzy Matching**

In [ ]:
# Install fuzzywuzzy
!pip install fuzzywuzzy[speedup]

import pandas as pd
from fuzzywuzzy import process
from IPython.display import display

# Load the datasets
nta = pd.read_csv('') #include NTA data
ntd = pd.read_csv('') #inclde NTD data

# Function to find the best match in another DataFrame and return both match and score
def get_best_match(row, matcher_df, matcher_column):
    # Ensure matcher_column is a list of strings
    options = matcher_df[matcher_column].dropna().unique().tolist()

    # Use FuzzyWuzzy to get the closest match and its score
    result = process.extractOne(row['station_name'], options)
    if result:
        match, score = result
        return match, score
    return None, None  # Return None for both match and score if no result

# Apply the function to find the best matches and store results in two new columns
nta[['matched_station', 'match_score']] = nta.apply(lambda x: get_best_match(x, nta, 'Facility Name'), axis=1, result_type='expand')

# Merge the datasets based on the matched names
merged_data = pd.merge(nta, ntd, left_on='matched_station', right_on='Facility Name', how='left')

# Select columns to include in the final dataset (adjust as needed)
final_data = merged_data[['station_name'] + [col for col in nta.columns if col not in ['station_name', 'matched_station', 'match_score']] + ['Facility ID', 'matched_station', 'match_score']]

# Display the merged DataFrame as an HTML table
display(final_data.head())


,station_name,Zipcode,Region,transit_mode,agency,line_name,line_station,year_stati,station_la,station_lo,Notes,Facility ID,matched_station,match_score
0,Burns Commons,53202,8,3,City of Milwaukee,The Hop Streetcar,The Hop Streetcar Burns Commons,2018,43.047819,-87.896260,NaN,16848,Burns Commons,100
1,Ogden at Astor,53202,8,3,City of Milwaukee,The Hop Streetcar,The Hop Streetcar Ogden at Astor,2018,43.048133,-87.900010,NaN,16847,Ogden at Astor Eastbound,90
2,Ogden/Jackson,53202,8,3,City of Milwaukee,The Hop Streetcar,The Hop Streetcar Ogden/Jackson,2018,43.048134,-87.904000,NaN,16846,Ogden/Jackson Eastbound,90
3,Jackson at Juneau,53202,8,3,City of Milwaukee,The Hop Streetcar,The Hop Streetcar Jackson at Juneau,2018,43.045781,-87.904768,NaN,16845,Jackson at Juneau Northbound,90
4,Cathedral Square,53202,8,3,City of Milwaukee,The Hop Streetcar,The Hop Streetcar Cathedral Square,2018,43.042576,-87.905047,NaN,16844,Cathedral Square,100


# **Export the Joined Data File**

In [ ]:
from google.colab import files

# Convert DataFrame to CSV and then download
final_data.to_csv('merged_dataset.csv', index=False)
files.download('merged_dataset.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>